In [7]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn import linear_model
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_log_error

In [8]:
# 读取player 的数据
player = pd.read_csv("../datafrom200/players.csv")
# x_list = player.columns[7:-5]
# x_list = ['Crossing', 'Finishing', 'Heading_Accuracy', 'Short_Passing', 'Volleys',
#        'Dribbling', 'Curve', 'FK_Accuracy', 'Long_Passing', 'Ball_Control',
#        'Acceleration', 'Sprint_Speed', 'Agility', 'Reactions', 'Balance',
#        'Shot_Power', 'Jumping', 'Stamina', 'Strength', 'Long_Shots',
#        'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties',
#        'Composure', 'Defensive_Awareness']
# x_list = ['Ball_Control', 'Sprint_Speed', 'Reactions', 'Stamina', 'Composure',
#        'Standing_Tackle', 'Sliding_Tackle']  # xgboost feature selection
x_list = ['Finishing', 'Short_Passing', 'Dribbling', 'Long_Passing',
       'Ball_Control', 'Acceleration', 'Sprint_Speed', 'Reactions', 'Balance',
       'Shot_Power', 'Stamina', 'Strength', 'Aggression', 'Vision',
       'Penalties', 'Standing_Tackle', 'Sliding_Tackle']  # random forest auto - feature selection
print(x_list)
X = player[x_list]
y = player["value"]
len(x_list)

['Finishing', 'Short_Passing', 'Dribbling', 'Long_Passing', 'Ball_Control', 'Acceleration', 'Sprint_Speed', 'Reactions', 'Balance', 'Shot_Power', 'Stamina', 'Strength', 'Aggression', 'Vision', 'Penalties', 'Standing_Tackle', 'Sliding_Tackle']


17

In [9]:
# 划分训练集 train, test : 0.8, 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21)
# X_validation, X_test, y_validation, y_test = train_test_split(X_test_val, y_test_val, test_size=0.5, random_state=23)

In [10]:
from sklearn.metrics import make_scorer, mean_squared_error
# 创建一个自定义的评分器，这个评分器返回RMSE
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# 使用make_scorer函数来将自定义评分器转换成一个合适的形式
rmse_scorer = make_scorer(rmse, greater_is_better=False)


In [11]:
# XGBoost
import xgboost as xgb
model = xgb.XGBRegressor()
params =  {'learning_rate': [0.3], 'max_depth': [3]}

# 传递自定义的评分器给 GridSearchCV
grid_search = GridSearchCV(model, params, scoring=rmse_scorer, cv=10, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# 打印最佳参数
print('Best parameters: ', grid_search.best_params_)
print('Best cross-validation score: ', - grid_search.best_score_)

Fitting 10 folds for each of 1 candidates, totalling 10 fits
[CV] END .....................learning_rate=0.3, max_depth=3; total time=   0.6s
[CV] END .....................learning_rate=0.3, max_depth=3; total time=   0.6s
[CV] END .....................learning_rate=0.3, max_depth=3; total time=   0.6s
[CV] END .....................learning_rate=0.3, max_depth=3; total time=   0.6s
[CV] END .....................learning_rate=0.3, max_depth=3; total time=   0.6s
[CV] END .....................learning_rate=0.3, max_depth=3; total time=   0.6s
[CV] END .....................learning_rate=0.3, max_depth=3; total time=   0.6s
[CV] END .....................learning_rate=0.3, max_depth=3; total time=   0.6s
[CV] END .....................learning_rate=0.3, max_depth=3; total time=   0.3s
[CV] END .....................learning_rate=0.3, max_depth=3; total time=   0.3s
Best parameters:  {'learning_rate': 0.3, 'max_depth': 3}
Best cross-validation score:  3214052.394501996


In [13]:
# Catboost
import catboost
model = catboost.CatBoostRegressor()
params = {'depth' : [16],
            # 'iterations' : [500, 1000, 1500],
            # 'learning_rate' : [0.01, 0.02, 0.03],
            # 'subsample' : [0.7, 0.8, 0.9, 1]
            }

grid_search = GridSearchCV(model, params, scoring=rmse_scorer, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# 打印最佳参数
print('Best parameters: ', grid_search.best_params_)
print('Best cross-validation score: ', - grid_search.best_score_)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Learning rate set to 0.055057
Learning rate set to 0.055058
Learning rate set to 0.055057
Learning rate set to 0.055057
Learning rate set to 0.055058
0:	learn: 8634509.2885629	total: 3.73s	remaining: 1h 2m 3s
0:	learn: 9192606.3267529	total: 3.74s	remaining: 1h 2m 20s
0:	learn: 8907653.5094060	total: 3.76s	remaining: 1h 2m 36s
0:	learn: 9291938.4391299	total: 3.79s	remaining: 1h 3m 3s
1:	learn: 8613042.9793237	total: 3.79s	remaining: 31m 30s
0:	learn: 9048041.6580073	total: 3.79s	remaining: 1h 3m 9s
1:	learn: 8403997.6270324	total: 7.05s	remaining: 58m 38s
1:	learn: 8792253.3846208	total: 7.09s	remaining: 58m 59s
1:	learn: 8928293.3726227	total: 7.18s	remaining: 59m 41s
1:	learn: 9027843.1335544	total: 7.28s	remaining: 1h 31s
2:	learn: 8648934.6439589	total: 7.28s	remaining: 40m 17s
2:	learn: 8745203.2892176	total: 7.37s	remaining: 40m 47s
2:	learn: 8367442.9775402	total: 7.6s	remaining: 42m 6s
2:	learn: 8121258.6231860	total: 

TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.

The exit codes of the workers are {SIGKILL(-9)}

In [14]:
import lightgbm as lgb
model = lgb.LGBMRegressor()
params = {'colsample_bytree': [0.7], 
          'learning_rate': [0.1], 
          'max_depth': [7], 
          'n_estimators': [900], 
          'num_leaves': [30], 
          'subsample': [0.7]
          }

grid_search = GridSearchCV(model, params, scoring=rmse_scorer, cv=10, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# 打印最佳参数
print('Best parameters: ', grid_search.best_params_)
print('Best cross-validation score: ', - grid_search.best_score_)

Fitting 10 folds for each of 1 candidates, totalling 10 fits
[CV] END colsample_bytree=0.7, learning_rate=0.1, max_depth=7, n_estimators=900, num_leaves=30, subsample=0.7; total time=   1.2s
[CV] END colsample_bytree=0.7, learning_rate=0.1, max_depth=7, n_estimators=900, num_leaves=30, subsample=0.7; total time=   1.2s
[CV] END colsample_bytree=0.7, learning_rate=0.1, max_depth=7, n_estimators=900, num_leaves=30, subsample=0.7; total time=   1.2s
[CV] END colsample_bytree=0.7, learning_rate=0.1, max_depth=7, n_estimators=900, num_leaves=30, subsample=0.7; total time=   1.2s
[CV] END colsample_bytree=0.7, learning_rate=0.1, max_depth=7, n_estimators=900, num_leaves=30, subsample=0.7; total time=   1.2s
[CV] END colsample_bytree=0.7, learning_rate=0.1, max_depth=7, n_estimators=900, num_leaves=30, subsample=0.7; total time=   1.3s
[CV] END colsample_bytree=0.7, learning_rate=0.1, max_depth=7, n_estimators=900, num_leaves=30, subsample=0.7; total time=   1.3s
[CV] END colsample_bytree=0.7

In [15]:
# 随机森林
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()
params = {'max_features': ['auto'], 
          'min_samples_split': [5], 
          'max_depth': [13], 
          'min_samples_leaf': [3]}

grid_search = GridSearchCV(model, params, scoring=rmse_scorer, cv=10, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# 打印最佳参数
print('Best parameters: ', grid_search.best_params_)
print('Best cross-validation score: ', - grid_search.best_score_)

Fitting 10 folds for each of 1 candidates, totalling 10 fits
[CV] END max_depth=13, max_features=auto, min_samples_leaf=3, min_samples_split=5; total time=   5.3s
[CV] END max_depth=13, max_features=auto, min_samples_leaf=3, min_samples_split=5; total time=   5.3s
[CV] END max_depth=13, max_features=auto, min_samples_leaf=3, min_samples_split=5; total time=   5.3s
[CV] END max_depth=13, max_features=auto, min_samples_leaf=3, min_samples_split=5; total time=   5.4s
[CV] END max_depth=13, max_features=auto, min_samples_leaf=3, min_samples_split=5; total time=   5.4s
[CV] END max_depth=13, max_features=auto, min_samples_leaf=3, min_samples_split=5; total time=   5.4s
[CV] END max_depth=13, max_features=auto, min_samples_leaf=3, min_samples_split=5; total time=   5.5s
[CV] END max_depth=13, max_features=auto, min_samples_leaf=3, min_samples_split=5; total time=   5.5s
[CV] END max_depth=13, max_features=auto, min_samples_leaf=3, min_samples_split=5; total time=   4.8s
[CV] END max_depth=13

In [16]:
# GBDT
from sklearn.ensemble import GradientBoostingRegressor
model = GradientBoostingRegressor()
params = {'n_estimators': [800], 
          'learning_rate': [0.1], 
          'max_depth': [5]}
grid_search = GridSearchCV(model, params, scoring=rmse_scorer, cv=10, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# 打印最佳参数
print('Best parameters: ', grid_search.best_params_)
print('Best cross-validation score: ', - grid_search.best_score_)

Fitting 10 folds for each of 1 candidates, totalling 10 fits
[CV] END ...learning_rate=0.1, max_depth=5, n_estimators=800; total time=  24.8s
[CV] END ...learning_rate=0.1, max_depth=5, n_estimators=800; total time=  24.8s
[CV] END ...learning_rate=0.1, max_depth=5, n_estimators=800; total time=  24.8s
[CV] END ...learning_rate=0.1, max_depth=5, n_estimators=800; total time=  24.9s
[CV] END ...learning_rate=0.1, max_depth=5, n_estimators=800; total time=  24.9s
[CV] END ...learning_rate=0.1, max_depth=5, n_estimators=800; total time=  25.0s
[CV] END ...learning_rate=0.1, max_depth=5, n_estimators=800; total time=  25.0s
[CV] END ...learning_rate=0.1, max_depth=5, n_estimators=800; total time=  24.9s
[CV] END ...learning_rate=0.1, max_depth=5, n_estimators=800; total time=  25.1s
[CV] END ...learning_rate=0.1, max_depth=5, n_estimators=800; total time=  25.5s
Best parameters:  {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 800}
Best cross-validation score:  3136969.999233779


In [17]:
# Adaboost
from sklearn.ensemble import AdaBoostRegressor
model = AdaBoostRegressor()
params = {'n_estimators': [150], 
          'loss': ['square'], 
          'learning_rate': [0.1]}
grid_search = GridSearchCV(model, params, scoring=rmse_scorer, cv=10, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# 打印最佳参数
print('Best parameters: ', grid_search.best_params_)
print('Best cross-validation score: ', - grid_search.best_score_)

Fitting 10 folds for each of 1 candidates, totalling 10 fits
[CV] END ...learning_rate=0.1, loss=square, n_estimators=150; total time=   3.3s
[CV] END ...learning_rate=0.1, loss=square, n_estimators=150; total time=   3.4s
[CV] END ...learning_rate=0.1, loss=square, n_estimators=150; total time=   3.4s
[CV] END ...learning_rate=0.1, loss=square, n_estimators=150; total time=   3.4s
[CV] END ...learning_rate=0.1, loss=square, n_estimators=150; total time=   3.4s
[CV] END ...learning_rate=0.1, loss=square, n_estimators=150; total time=   3.4s
[CV] END ...learning_rate=0.1, loss=square, n_estimators=150; total time=   3.5s
[CV] END ...learning_rate=0.1, loss=square, n_estimators=150; total time=   3.5s
[CV] END ...learning_rate=0.1, loss=square, n_estimators=150; total time=   3.5s
[CV] END ...learning_rate=0.1, loss=square, n_estimators=150; total time=   3.6s
Best parameters:  {'learning_rate': 0.1, 'loss': 'square', 'n_estimators': 150}
Best cross-validation score:  4414311.792160345


In [18]:
# # Adaboost
# from sklearn.ensemble import AdaBoostRegressor
# i = 10
# clf = AdaBoostRegressor(n_estimators=i)
# clf.fit(X_train, y_train)
# print("R", clf.score(X_validation, y_validation))
# print("RMSE", np.sqrt(mean_squared_error(clf.predict(X_validation), y_validation)))
# print("MAE", mean_absolute_error(clf.predict(X_validation), y_validation))
# print("RMSLE", mean_squared_log_error(clf.predict(X_validation), y_validation))

In [19]:
# list_adaboost # adaboost: 10, 0.6945326685800794
# n_list_lbm = np.array(list_lightgbm) # leaves, depth, n, score : 31, 19, 500, 0.8007898902056965
# list_randomforest # [80, 0.7839175337533079]
# list_gbdt # [n = 600, 0.8264955831199144]
# catboost  defalt: 0.8027687798692258
# list_xgboost # [60, 20, 0.7812782293507701]

# Adaboost
 0.7678737783884331 R    

 4334351.772382652 RMSE 

2034372.1610311233 MAE   

 1.4164760588279637 RMSLE

# Xgboost
0.8868637133128766 R

3025958.826846494 RMSE 

923966.9140471614 MAE 

0.15334408868795865 RMSLE

# lgb
0.8948018678333571 R 

2917870.8720710855 RMSE 

863827.241753701 MAE 

# 随机森林
0.8810732244650263 R 

3102429.1779398853 RMSE 

971012.3118455497 MAE 

0.15934332311326996 RMSLE 

# GBDT
0.900796741780507 R 

2833511.7791834464 RMSE 

948986.9158787137 MAE 

# Catboost
0.8959925992883182 R 

2901310.272421131 RMSE 

879135.1904306137 MAE 